In [2]:
import joblib
import os
import numpy as np
import random
import pickle
from scipy import signal as scipysignal
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from loguru import logger

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [5]:
def data_generate(trdata_path):
    # 读取数据
    pkl_file = open(os.path.join(trdata_path, 'data.pkl'), "rb")
    eeg = pickle.load(pkl_file)
    data = eeg['data'][:64,:]
    trigger = eeg['data'][-1,:]
    index = np.where(trigger != 0)[0]
    triggerdense = trigger[index]
    SRATE = eeg['srate']
    MICHS = np.array(range(17, 49))

    # left-mi 索引
    l_indx = index[triggerdense == 1]
    # left-mi 标签
    l_labels = triggerdense[triggerdense == 1]

    # right-mi 索引
    r_indx = index[triggerdense == 2]
    # right-mi 标签
    r_labels = triggerdense[triggerdense == 2]

    lr_indx = np.concatenate((l_indx, r_indx))
    # idle 标签
    i_labels = np.zeros_like(r_labels)+7
    # idle 索引
    i_indx = np.array(random.sample(list(lr_indx), len(i_labels))).astype(np.int32)

    data = data[MICHS,:]     #最后五个通道为心电通道等

    fs = SRATE / 2
    FLTNUM = scipysignal.firwin(SRATE * 3 + 1, np.array([7, 30]) / fs, pass_zero='bandpass')  # 带通滤波

    # 构造left-mi训练集
    l_epochdata = np.zeros([len(l_labels),data.shape[0],4 * SRATE])
    for i, point in enumerate(l_indx):
        epoch = data[:, point + int(0.5*SRATE):point + int(0.5*SRATE) + 4 * SRATE]  # 截取该段信号
        epoch = scipysignal.filtfilt(FLTNUM, 1, epoch, padlen=len(epoch)-1)
        l_epochdata[i, :, :] = epoch

    # 构造right-mi训练集
    r_epochdata = np.zeros([len(r_labels),data.shape[0],4 * SRATE])
    for i, point in enumerate(r_indx):
        epoch = data[:, point + int(0.5*SRATE):point + int(0.5*SRATE) + 4 * SRATE]  # 截取该段信号
        epoch = scipysignal.filtfilt(FLTNUM, 1, epoch, padlen=len(epoch)-1)
        r_epochdata[i, :, :] = epoch

    # 构造idle训练集
    i_epochdata = np.zeros([len(i_labels),data.shape[0],4 * SRATE])
    for i, point in enumerate(i_indx):
        epoch = data[:, point - int(4.5*SRATE): point - int(0.5*SRATE)]  # 截取该段信号
        epoch = scipysignal.filtfilt(FLTNUM, 1, epoch, padlen=len(epoch)-1)
        i_epochdata[i, :, :] = epoch
    print(i_epochdata.shape)

    return [l_epochdata, l_labels], [r_epochdata, r_labels], [i_epochdata, i_labels]


In [3]:
pipe = Pipeline([('csp',CSP()),('svc', SVC())])

In [8]:
mitrain_path = os.path.abspath('.')
trdata_path = os.path.join(mitrain_path, 'Trainingdata', 'S1')

In [10]:
trdata_path

'/home/twjia/turing_new_local_beta/Algorithm/MItrain/Trainingdata/S1'

In [11]:
l_list, r_list, i_list = data_generate(trdata_path)

(100, 32, 1000)


In [18]:
len(l_list)
print(l_list[0].shape)
print(l_list[1].shape)


(100, 32, 1000)
(100,)


In [19]:
X = np.concatenate((l_list[0], r_list[0]),axis=0)
y = np.concatenate((l_list[1], r_list[1]),axis=0)

In [25]:
X_csp = pipe.fit(X,y)

Computing rank from data with rank=None
    Using tolerance 2.5e-05 (2.2e-16 eps * 32 dim * 3.5e+09  max singular value)
    Estimated rank (mag): 32
    MAG: rank 32 computed from 32 data channels with 0 projectors
Reducing data rank from 32 -> 32
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.6e-05 (2.2e-16 eps * 32 dim * 3.6e+09  max singular value)
    Estimated rank (mag): 32
    MAG: rank 32 computed from 32 data channels with 0 projectors
Reducing data rank from 32 -> 32
Estimating covariance using EMPIRICAL
Done.


In [26]:
X_csp.shape

AttributeError: 'Pipeline' object has no attribute 'shape'